<a href="https://colab.research.google.com/github/Ryan-RE-Wang/BCI_project/blob/main/load_data_from_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from collections import Counter
from datetime import datetime
import time
import os, sys

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, cohen_kappa_score, accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 4.4 MB/s 


In [5]:
sys.path.append('/content/drive/Shareddrives/BCI/sleep_study/sleep_study')
import sleep_study as ss

# Useful Functions

In [6]:
def get_metrics(y, y_hat, verbose=True):
    cf = confusion_matrix(y, y_hat)
    ncf = 100*confusion_matrix(y, y_hat, normalize='true') # percentage
    precision,recall,mf,support = precision_recall_fscore_support(y,y_hat,average='macro')
    kp = cohen_kappa_score(y, y_hat)
    acc = 100*accuracy_score(y, y_hat) # percentage
    
    if verbose:
        print('\nConfusion Matrix')
        print(cf)
        print('\nNormalized Confusion Matrix')
        print(np.round(ncf))
        print('Accuracy: ', acc)
        print('Precision: ',precision)
        print('Recall: ',recall)
        print('Macro-F1 Score: ',mf)
        print('Kohen Kappa: ',kp)
        
    return {'confusion matrix': cf,
            'normalized confusion matrix': ncf,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'Macro-F1 Score': mf,
            'Kohen Kappa': kp}

In [7]:
def print_all_metrics(res, keys):
    ncf_name = 'normalized confusion matrix'
    
    for i, k in enumerate(keys):
        all_metrics = res[k]
        num_trials = len(all_metrics)
                
        means = pd.DataFrame(data = np.round(np.mean([all_metrics[rr][ncf_name] for rr in range(num_trials)], axis=0), 1), dtype='str')
        stds = pd.DataFrame(data = np.round(np.std([all_metrics[rr][ncf_name] for rr in range(num_trials)], axis=0), 1), dtype='str')
        tbl = (means + " " + u"\u00B1" + " " + stds).to_numpy()
        print(ncf_name)
        print(tbl) # I want normalized CF to print nicely with mean \pm standard deviation.

        for j in ['accuracy', 'precision', 'recall', 'Macro-F1 Score', 'Kohen Kappa']:
            m = str(np.round(np.mean([all_metrics[rr][j] for rr in range(num_trials)]), 3))
            s = str(np.round(np.std([all_metrics[rr][j] for rr in range(num_trials)]), 3))
            print(j, ":", m + " " + u"\u00B1" + " " + s)
        print( )

In [8]:
def get_npz_file_names(path):
    files = []
    with os.scandir(path) as it:
        for entry in it:
            if (entry.name.endswith('.npz') or entry.name.endswith('.NPZ')) and entry.is_file():
                files.append(path+entry.name)
    return files

In [9]:
def load_features(fn, verbose=False):   
    X = np.load(fn, allow_pickle=True)
    features, labels = X["features"], X["labels"]
    if verbose:
        print(fn)
        print(features.shape, labels.shape)
        
    return features, labels

In [10]:
def train_test_classifier(idxs, data, clf, verbose=True):
    
    train_idx, test_idx = idxs
    X, y = data
    
    X_train, X_test = X[train_idx,:], X[test_idx,:]
    y_train, y_test = y[train_idx], y[test_idx]
    
    if verbose:
        print('train set class distribution', sorted(Counter(y_train).items()))
        print('test set class distribution', sorted(Counter(y_test).items()))

    t = time.time()
    clf.fit(X_train, y_train)

    if verbose:
        print('training took', np.around((time.time()-t)/60, 2), 'minutes.')

    y_hat = rf.predict(X_test)

    metrics = get_metrics(y_test, y_hat, verbose=False)

    if verbose:
        print(metrics)
        print( )

    return metrics, y_hat

# Train and test classifier

In [16]:
seed = 0 # for reproducibility
num_trials = 3

rf = RandomForestClassifier(n_estimators=100, random_state=seed, bootstrap=True, n_jobs=-1)
cv = StratifiedKFold(n_splits=num_trials, shuffle=True, random_state=seed)

feature_files = get_npz_file_names('/content/drive/Shareddrives/BCI/BCI_Data/Sleep_Data/')[0:1]
print(feature_files)

['/content/drive/Shareddrives/BCI/BCI_Data/Sleep_Data/wavelet_features20743_15763.npz']


In [17]:
features, labels = load_features(fn)

# features are waveform 
# labels are the icd_code